# **``Dimensionality Reduction Techniques``**

---

### 1. [SVD](#SVD__(Singular__Value__Decomposition))
- #### [Using Numpy](#Using_Numpy)
### 2. [Truncated SVD](#A)
### 3. [ABC](#B)

In [1]:
import os
import sys
import logging

logging.basicConfig(filename="SA1_PCA.log",
                    filemode='w',
                    level=logging.INFO,
                    format="%(asctime)s : %(levelname)s : %(message)s")

try :
    logging.info("#### Packages import ####")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import sklearn
    from sklearn import datasets
except ImportError as ie:
    # Output expected ImportErrors
    logging.error(msg=ie.__class__.__name__  + " :: Missing Package --> " + ie.name)
except Exception as exception:
    # Output unexpected Exceptions
    logging.info("#### Exceptions other than ModuleImportError ####")
    logging.log(msg=(exception, False))
    logging.log(msg=exception.__class__.__name__ + " :: " + exception.name)

%matplotlib inline

# **``SVD__(Singular__Value__Decomposition)``**

#### **Lets first understand how SVD works using a dummy dataset, then we will apply the same concept on Breast Cancer Dataset. There are multiple ways by which we can apply SVD on our dataset. First, I'll demonstrate the same using Numpy then via Sklearn.**

## **``Using Numpy``**

In [2]:
X = pd.DataFrame({'col1':[9,4,7,4],
                  'col2':[3,2,1,2]})

In [3]:
X

,col1,col2
0,9,3
1,4,2
2,7,1
3,4,2


In [4]:
X.shape, X.ndim

((4, 2), 2)

In [5]:
from numpy.linalg import svd

In [6]:
U,S,VT = svd(X,full_matrices=True,compute_uv=True,hermitian=False)

In [7]:
pd.DataFrame(U)

,0,1,2,3
0,-0.711633,-0.113179,-0.642945,-0.259597
1,-0.331229,-0.466058,0.650505,-0.499920
2,-0.523597,0.743485,0.385767,0.155758
3,-0.331229,-0.466058,0.121029,0.811436


In [8]:
Sigma = np.zeros((X.shape[0],X.shape[1]))
Sigma

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [9]:
Sigma[:X.shape[1],:X.shape[1]] = np.diag(S)
Sigma

array([[13.32885697,  0.        ],
       [ 0.        ,  1.53021959],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [10]:
pd.DataFrame(VT)

,0,1
0,-0.954298,-0.298856
1,0.298856,-0.954298


### **``How to re-construct the main dataset from U, Sigma and VT?``**

#### **Way-1 : Using Dot Product**

In [11]:
np.dot(U,np.dot(Sigma,VT))

array([[9., 3.],
       [4., 2.],
       [7., 1.],
       [4., 2.]])

#### **Way-2 : Using Matrix Multiplication**

In [12]:
U @ Sigma @ VT

array([[9., 3.],
       [4., 2.],
       [7., 1.],
       [4., 2.]])

# **`2. Truncated SVD`**
#### **Contrary to PCA, this estimator does not center the data before computing the singular value decomposition. This means it can work with sparse matrices efficiently.**

#### **In particular, truncated SVD works on term count/tf-idf matrices as returned by the vectorizers in :mod:`sklearn.feature_extraction.text`. In that context, it is known as `latent semantic analysis (LSA)`.**

#### **This estimator supports two algorithms: a fast randomized SVD solver, and a "naive" algorithm that uses ARPACK as an eigensolver on `X * X.T` or `X.T * X`, whichever is more efficient.**

## **``Using Sklearn``**

In [13]:
from sklearn.decomposition import TruncatedSVD

In [14]:
tsvd = TruncatedSVD(n_components=2)

In [15]:
X2 = pd.DataFrame({'col1':[9,4,7,4],
                   'col2':[3,2,1,2],
                   'col3':[5,6,7,1]})

In [16]:
X2.shape, X2.ndim

((4, 3), 2)

In [17]:
X_transf = tsvd.fit_transform(X2)

#### **Variance of every component**

In [18]:
vect_magnitude = tsvd.explained_variance_                 ## The magnitude or amount of variance in every component
vect_magnitude

array([6.42468871, 3.2616684 ])

##### **``Here, explained_variance_ represents the variance of both the components.``**

In [19]:
np.var(X_transf[:,0]), np.var(X_transf[:,1])

(6.4246887130369394, 3.261668395175181)

#### **Percentage of variations explained by components**

In [20]:
expln_var_ratio = tsvd.explained_variance_ratio_           ## Percentage of variation explained by each component
expln_var_ratio

array([0.63064429, 0.32016377])

##### **``Here, explained_variance_ratio is the ratio of the variances in the components and the sum of the variance in the original dataset.``**

In [21]:
np.var(X_transf,axis=0)/np.sum(np.var(X2,axis=0))

array([0.63064429, 0.32016377])

#### **Total percentage of variations explained by components**

In [22]:
tot_var_expln = np.sum((expln_var_ratio))*100              ## Total variation explained by both the components 
print('Total variation explained by the components is {0:.3f}%'.format(tot_var_expln))

Total variation explained by the components is 95.081%


##### **``It is just the addition of components individual percentage of variances.``**

In [23]:
np.sum(np.var(X_transf,axis=0)/np.sum(np.var(X2,axis=0)))

0.9508080597018033

#### **Number of components**

In [24]:
tsvd.n_components            ## Number of generated components            

2

#### **VT**

In [25]:
VT_x2 = tsvd.components_     ## These are the values of VT's i.e. the EIGEN VECTORS
VT_x2

array([[ 0.75687368,  0.23209938,  0.61095999],
       [ 0.53103415,  0.32653333, -0.7819071 ]])

### **So, one thing to understand here is that the above VT_x2 is representing the two eigen vectors whose amount of variation or magnitude(means the eigen value) is represented by singular_values_.**

In [26]:
tsvd.singular_values_        ## These are the values of the sigma's i.e. the EIGEN VALUES

array([16.60761964,  3.62068869])

In [27]:
U_Sigma_x2 = X_transf        ## The fit_transform function returns us the product of U and Sigma's
U_Sigma_x2

array([[10.56296122,  1.84937181],
       [ 7.15745341, -1.91423937],
       [ 9.80693507, -1.42957735],
       [ 4.10265348,  1.99529615]])

### **Here, one point which needs to be understand is that SVD is nothing but breaking the higher rank dataset into one set(U,Singma and VT) of Rank-1 matrices or closest approximation of main dataset is achieved by the addition of more than one set(U,Sigma and VT) of Rank-1 matrices which can yield a higher rank dataset.**

### **Fit_transform returns the product of U and Sigma, so if we want to reconstruct our main dataset then we just need to peform its matrix multiplication with the VT's or components_.**

### **``How to re-construct the main dataset from U, Sigma and VT?``**

#### **Way-1 : Using Matrix Multiplication**

In [28]:
retrv_x2 = pd.DataFrame(U_Sigma_x2 @ VT_x2).apply(lambda val : np.round(val,4))
retrv_x2

,0,1,2
0,8.9769,3.0555,5.0075
1,4.4008,1.0362,5.8697
2,6.6635,1.8094,7.1094
3,4.1648,1.6038,0.9464


#### **Way-2 : Using Dot Product**

In [29]:
pd.DataFrame(np.dot(U_Sigma_x2, VT_x2)).apply(lambda val : np.round(val,4))

,0,1,2
0,8.9769,3.0555,5.0075
1,4.4008,1.0362,5.8697
2,6.6635,1.8094,7.1094
3,4.1648,1.6038,0.9464


### **We have reterieved X2 that was our dataset of shape (4,3) with 12 elements and it is the closest approximate with two components having 8 elements.**

### **Here, it might feels like we have saved a very little space but imagine if we have 1000 x 1000 dataset then in such as case we will only consume space of 2000 elements instead of 10,00,000.**

In [30]:
X2

,col1,col2,col3
0,9,3,5
1,4,2,6
2,7,1,7
3,4,2,1


In [31]:
X3 = pd.DataFrame({'col1':[9,0,0,4],
                   'col2':[0,2,1,0],
                   'col3':[0,6,0,1],
                   'col4':[0,0,3,9]})

In [32]:
from scipy.sparse.linalg import svds

In [33]:
tsvd_rand2 = svds(A=X3,k=2,solver='lobpcg')

In [34]:
tsvd_rand2

(array([[ 0.79503236, -0.57680399],
        [-0.1934866 , -0.05658798],
        [-0.28605322, -0.17890175],
        [-0.49865823, -0.79504033]]),
 array([ 7.64330878, 11.42894391]),
 array([[ 0.67518643, -0.08805433, -0.21712819, -0.69944626],
        [-0.73247338, -0.02555597, -0.09927148, -0.67303403]]))

In [35]:
tsvd_rand2 = np.linalg.svd(X3)

In [36]:
U3, S3, VT3 = tsvd_rand2
U3, S3, VT3

(array([[-5.76803991e-01,  7.95032363e-01,  1.23712813e-01,
         -1.41123481e-01],
        [-5.65879774e-02, -1.93486597e-01,  9.79469615e-01,
         -1.06244511e-04],
        [-1.78901749e-01, -2.86053223e-01, -6.69453278e-02,
         -9.38981385e-01],
        [-7.95040326e-01, -4.98658231e-01, -1.44404766e-01,
          3.13684732e-01]]),
 array([11.42894391,  7.64330878,  6.24329347,  0.99013065]),
 array([[-0.73247338, -0.02555597, -0.09927148, -0.67303403],
        [ 0.67518643, -0.08805433, -0.21712819, -0.69944626],
        [ 0.08581949,  0.3030442 ,  0.91817131, -0.24033451],
        [-0.01552563, -0.9485555 ,  0.31616763,  0.00628042]]))

In [37]:
sigma = np.zeros((X3.shape[0],X3.shape[1]))
sigma = np.diag(S3)
sigma

array([[11.42894391,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  7.64330878,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  6.24329347,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.99013065]])

In [38]:
pd.DataFrame(U3 @ (sigma @ VT3)).apply(lambda val : np.round(val,2))

,0,1,2,3
0,9.0,0.0,-0.0,-0.0
1,0.0,2.0,6.0,0.0
2,-0.0,1.0,0.0,3.0
3,4.0,-0.0,1.0,9.0


In [39]:
X3

,col1,col2,col3,col4
0,9,0,0,0
1,0,2,6,0
2,0,1,0,3
3,4,0,1,9


## **``Matrix-Free Solvers``**

In [40]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
from sklearn.random_projection import sparse_random_matrix

In [41]:
X_sp = sparse_random(2000, 2000, density=0.01, format='csr',random_state=42)

In [42]:
X_sp.shape

(2000, 2000)

In [43]:
X_sp.ndim

2

In [44]:
X_sp.data.nbytes, X_sp.indptr.nbytes, X_sp.indices.nbytes

(320000, 8004, 160000)

In [45]:
X_sp.data.nbytes + X_sp.indptr.nbytes + X_sp.indices.nbytes

488004

In [46]:
X_sp_matrix = pd.DataFrame.sparse.from_spmatrix(X_sp)
X_sp_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
X_sp_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 2000 entries, 0 to 1999
dtypes: Sparse[float64, 0](2000)
memory usage: 468.9 KB


In [48]:
X_sp_matrix.ndim

2

In [49]:
X_sp_matrix.shape

(2000, 2000)

In [50]:
import scipy

In [51]:
X_sp_recons = scipy.sparse.csr_matrix(X_sp_matrix)

In [52]:
X_sp_recons.data.nbytes + X_sp_recons.indptr.nbytes + X_sp_recons.indices.nbytes

488004

In [53]:
X_sp_recons.sum()

19988.697889128358

In [54]:
sum(X_sp_matrix)

1999000

In [55]:
import timeit

In [61]:
U_sp_matrix, S_sp_matrix, VT_sp_matrix = np.linalg.svd(X_sp_matrix)

%timeit np.linalg.svd(X_sp_matrix)      ## Commenting this line as running it will consume a lot of memory 

In [63]:
U_sp_matrix.shape, S_sp_matrix.shape, VT_sp_matrix.shape

((2000, 2000), (2000,), (2000, 2000))

### **For a matrix of shape (2000, 2000), below are the timings:**
#### **12.2 s ± 429 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)**

In [ ]:
## Commenting this cell before running this notebook as it will consume a lot of memory 
%timeit scipy.sparse.linalg.svds(X_sp_new,k=10)

### **For a matrix of shape (2000, 2000), below are the timings:**
#### **191 ms ± 18.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)**

In [ ]:
%timeit scipy.sparse.linalg.svds(X_sparse,k=1999)

In [59]:
scipy.sparse.linalg.svds(X_sparse,k=1999)

(array([[-0.02063183, -0.01205017,  0.00514028, ..., -0.00763698,
          0.07987386, -0.03566112],
        [-0.03635916, -0.06628678, -0.01556633, ...,  0.01027269,
         -0.01461515, -0.02279414],
        [-0.02261808,  0.00096161, -0.03427344, ..., -0.00517224,
          0.00167156, -0.02144013],
        ...,
        [ 0.00040521,  0.01201483,  0.0076881 , ..., -0.01019085,
          0.01659742, -0.02333169],
        [-0.00310962, -0.00054107,  0.0250472 , ...,  0.00464826,
         -0.00642858, -0.02367788],
        [ 0.01444094, -0.01494037, -0.00525141, ...,  0.03656955,
          0.02193873, -0.02194574]]),
 array([2.91811636e-03, 6.07626807e-03, 6.97272183e-03, ...,
        5.33341589e+00, 5.34790190e+00, 1.07026374e+01]),
 array([[ 0.01276915,  0.00144004,  0.00416085, ...,  0.0009266 ,
          0.01109519,  0.01228572],
        [-0.01097065, -0.00016862,  0.00338394, ...,  0.00138614,
          0.02244893, -0.01034522],
        [ 0.03020293, -0.01424551, -0.01348938, ..

In [60]:
from scipy.sparse.linalg import svds

In [61]:
breast_cancer = datasets.load_breast_cancer()

In [62]:
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [63]:
cancer_df = pd.concat([pd.DataFrame(breast_cancer.data,columns=breast_cancer.feature_names),
                       pd.DataFrame(breast_cancer.target,columns=['Label'])],axis=1)

cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### **Reference Links**

https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/

https://www.analyticsvidhya.com/blog/2019/08/5-applications-singular-value-decomposition-svd-data-science/

https://www.youtube.com/watch?v=46Hpy4FiGls&list=PLMrJAkhIeNNSVjnsviglFoY2nXildDCcv&index=10